In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
transactions_df = pd.read_excel('../OneDrive_1_2024-2-19/transactions.xlsx')
actions_df = pd.read_excel('../OneDrive_1_2024-2-19/actions.xlsx')

In [3]:
transactions_df

,client_id,transaction_id,transaction_date,product_quantity,gross_amount_euro,product_category,product_subcategory,product_style
0,c17974679,t11288118,2020-07-25,1,3044,Women Ready-to-Wear,Unknown,Casual Style
1,c17974679,t93389142,2020-06-20,1,569,Women Accessory,Unknown,Stylish Fashion
2,c17974679,t62924895,2021-01-13,1,7727,Women Bags,Unknown,Leather
3,c17974679,t93389142,2020-06-20,1,3954,Women Ready-to-Wear,Unknown,Casual Style
4,c17974679,t33368160,2021-05-03,1,1541,Women Ready-to-Wear,bottom segment,Casual Style
...,...,...,...,...,...,...,...,...
54633,c36938015,t19758562,2021-07-03,1,2220,Woman Shoes,Unknown,Easy Day Style
54634,c19544295,t83687427,2021-12-27,1,6472,Women Bags,Unknown,Leather
54635,c07447234,t31595753,2021-05-27,1,111,Women Accessory,bottom segment,Fashion Style
54636,c08145778,t79698899,2021-11-02,1,1220,Women Accessory,Unknown,Stylish Fashion


In [4]:
most_popular_product_category = transactions_df['product_category'].value_counts().idxmax()
most_popular_product_category
transactions_df.loc[transactions_df['product_category'] == 'Unknown', 'product_category'] = most_popular_product_category

In [5]:
transactions_df.columns

Index(['client_id', 'transaction_id', 'transaction_date', 'product_quantity',
       'gross_amount_euro', 'product_category', 'product_subcategory',
       'product_style'],
      dtype='object')

In [6]:
merged_df = pd.merge(transactions_df, actions_df, on='client_id')

merged_df['transaction_date'] = pd.to_datetime(merged_df['transaction_date'])
merged_df['action_end_date'] = pd.to_datetime(merged_df['action_end_date'])

rates = []
for client_id, group in merged_df.groupby('client_id'):
    for category, category_group in group.groupby('product_category'):
        total_transactions = len(category_group)
        transactions_within_13_days = len(category_group[category_group['transaction_date'] <= category_group['action_end_date'] + pd.Timedelta(days=13)])

        if total_transactions > 0:
            rate = transactions_within_13_days / total_transactions
        else:
            rate = 0

        rates.append({
            'client_id': client_id,
            'product_category': category,
            'transactions_within_13_days': transactions_within_13_days,
            'total_transactions': total_transactions,
            'rate': rate
        })


rates_df = pd.DataFrame(rates)
average_rates_per_category = rates_df.groupby('product_category')['rate'].mean().reset_index()
print(average_rates_per_category)

             product_category      rate
0                       Child  0.518098
1                       House  0.509227
2                     Jewelry  0.641272
3               Men Accessory  0.495522
4                    Men Bags  0.566141
5           Men Ready-to-Wear  0.557108
6                   Men Shoes  0.586602
7     Men Small Leather Goods  0.515928
8                     Watches  0.720430
9                 Woman Shoes  0.595019
10            Women Accessory  0.600500
11                 Women Bags  0.634092
12        Women Ready-to-Wear  0.584419
13  Women Small Leather Goods  0.569341


In [7]:
merged_df = pd.merge(transactions_df, actions_df, on='client_id')
merged_df['transaction_date'] = pd.to_datetime(merged_df['transaction_date'])
merged_df['action_end_date'] = pd.to_datetime(merged_df['action_end_date'])
rates = []

for client_id, group in merged_df.groupby('client_id'):
    for category, category_group in group.groupby('product_category'):
        total_transactions = len(category_group)
        transactions_within_event_days = len(category_group[category_group['transaction_date'] <= category_group['action_end_date'] + pd.Timedelta(days=13)])

        if total_transactions > 0:
            rate = transactions_within_event_days / total_transactions
        else:
            rate = 0

        rates.append({
            'client_id': client_id,
            'product_category': category,
            'transactions_within_event_days': transactions_within_event_days,
            'total_transactions': total_transactions,
            'rate': rate
        })

rates_df = pd.DataFrame(rates)
print(rates_df)


       client_id           product_category  transactions_within_event_days  \
0      c00029531            Women Accessory                               3   
1      c00029531  Women Small Leather Goods                               2   
2      c00089184        Women Ready-to-Wear                               5   
3      c00107911                Woman Shoes                               0   
4      c00107911            Women Accessory                               0   
...          ...                        ...                             ...   
12103  c99936245                Woman Shoes                               2   
12104  c99936245            Women Accessory                               1   
12105  c99936245        Women Ready-to-Wear                               1   
12106  c99976540            Women Accessory                               0   
12107  c99976540                 Women Bags                               0   

       total_transactions      rate  
0            

In [8]:
print(rates_df['transactions_within_event_days'].sum()/rates_df['total_transactions'].sum())

0.5484156512358058


# Start from here!

In [9]:
duplicate_transactions = transactions_df[transactions_df.duplicated(subset=['transaction_id'], keep=False)]

if len(duplicate_transactions) > 0:
    print("Duplicate transactions found:")
    print(duplicate_transactions.nunique())
else:
    print("No duplicate transactions found.")

Duplicate transactions found:
client_id              2501
transaction_id         8997
transaction_date        820
product_quantity         11
gross_amount_euro      7658
product_category         14
product_subcategory       3
product_style            36
dtype: int64


In [10]:
transactions_df['unique_transaction_id'] = range(len(transactions_df))

In [11]:
transactions_df

,client_id,transaction_id,transaction_date,product_quantity,gross_amount_euro,product_category,product_subcategory,product_style,unique_transaction_id
0,c17974679,t11288118,2020-07-25,1,3044,Women Ready-to-Wear,Unknown,Casual Style,0
1,c17974679,t93389142,2020-06-20,1,569,Women Accessory,Unknown,Stylish Fashion,1
2,c17974679,t62924895,2021-01-13,1,7727,Women Bags,Unknown,Leather,2
3,c17974679,t93389142,2020-06-20,1,3954,Women Ready-to-Wear,Unknown,Casual Style,3
4,c17974679,t33368160,2021-05-03,1,1541,Women Ready-to-Wear,bottom segment,Casual Style,4
...,...,...,...,...,...,...,...,...,...
54633,c36938015,t19758562,2021-07-03,1,2220,Woman Shoes,Unknown,Easy Day Style,54633
54634,c19544295,t83687427,2021-12-27,1,6472,Women Bags,Unknown,Leather,54634
54635,c07447234,t31595753,2021-05-27,1,111,Women Accessory,bottom segment,Fashion Style,54635
54636,c08145778,t79698899,2021-11-02,1,1220,Women Accessory,Unknown,Stylish Fashion,54636


In [12]:
merged_df = pd.merge(transactions_df, actions_df, on='client_id', how='left')

In [13]:
# sorting
merged_df.sort_values(by='action_end_date', ascending=False, inplace=True)

# keep the recent one
unique_transactions_df = merged_df.drop_duplicates(subset=['unique_transaction_id'], keep='first')

unique_transactions_df.reset_index(drop=True, inplace=True)

print(unique_transactions_df)

       client_id transaction_id transaction_date  product_quantity  \
0      c03877908      t15280476       2020-03-12                 1   
1      c03877908      t13409808       2020-05-15                 1   
2      c03877908      t13457490       2020-10-10                 1   
3      c03877908      t86627597       2020-05-09                 1   
4      c03877908      t13409808       2020-05-15                 1   
...          ...            ...              ...               ...   
54633  c36938015      t19758562       2021-07-03                 1   
54634  c19544295      t83687427       2021-12-27                 1   
54635  c07447234      t31595753       2021-05-27                 1   
54636  c08145778      t79698899       2021-11-02                 1   
54637  c54482513      t94462221       2022-01-12                 1   

       gross_amount_euro     product_category product_subcategory  \
0                   6461  Women Ready-to-Wear             Unknown   
1                   5

In [14]:
# are there any duplicates in the transactions?

duplicate_transactions = unique_transactions_df[unique_transactions_df.duplicated(subset=['unique_transaction_id'], keep=False)]

if len(duplicate_transactions) > 0:
    print("Duplicate transactions found:")
    print(duplicate_transactions.nunique())
else:
    print("No duplicate transactions found.")

No duplicate transactions found.


In [15]:
from datetime import datetime, timedelta

In [16]:
merged_df['transaction_date'] = pd.to_datetime(merged_df['transaction_date'])
merged_df['action_end_date'] = pd.to_datetime(merged_df['action_end_date'])

# compute rates by client, product category
rates = []
total_transaction_during_event_count = 0
for client_id, group in unique_transactions_df.groupby('client_id'):
    for category, category_group in group.groupby('product_category'):
        total_transactions = len(category_group)
        transactions_within_event_days = len(category_group[(category_group['transaction_date'] <= category_group['action_end_date']+timedelta(days=7)) &
         (category_group['transaction_date'] >= category_group['action_start_date']-timedelta(days=7)) &
          (category_group['client_is_invited']==1)])

        total_transaction_during_event_count = total_transaction_during_event_count + transactions_within_event_days

        if total_transactions > 0:
            rate = transactions_within_event_days / total_transactions
        else:
            rate = 0

        rates.append({
            'client_id': client_id,
            'product_category': category,
            'transactions_within_event_days': transactions_within_event_days,
            'total_transactions': total_transactions,
            'rate': rate
        })

print(f'avg stimulation by events: {total_transaction_during_event_count/len(unique_transactions_df)}')

rates_df = pd.DataFrame(rates)
average_rates_per_category = rates_df.groupby('product_category')['rate'].mean().reset_index()
print(average_rates_per_category)

avg stimulation by events: 0.060617152897250996
             product_category      rate
0                       Child  0.021945
1                          HC  0.000000
2                       House  0.037763
3                     Jewelry  0.070060
4               Men Accessory  0.065554
5                    Men Bags  0.046970
6           Men Ready-to-Wear  0.045955
7                   Men Shoes  0.076806
8     Men Small Leather Goods  0.037238
9                     Watches  0.081081
10                Woman Shoes  0.065215
11            Women Accessory  0.050989
12                 Women Bags  0.085031
13        Women Ready-to-Wear  0.060282
14  Women Small Leather Goods  0.050939


In [17]:
average_rates_per_category['product_category'].unique()

array(['Child', 'HC', 'House', 'Jewelry', 'Men Accessory', 'Men Bags',
       'Men Ready-to-Wear', 'Men Shoes', 'Men Small Leather Goods',
       'Watches', 'Woman Shoes', 'Women Accessory', 'Women Bags',
       'Women Ready-to-Wear', 'Women Small Leather Goods'], dtype=object)

In [18]:
merged_df.nunique()

client_id                   13884
transaction_id              35452
transaction_date             1480
product_quantity               14
gross_amount_euro           10227
product_category               15
product_subcategory             3
product_style                  38
unique_transaction_id       54638
action_id                     385
action_type_label               3
action_subcategory_label        2
action_start_date             258
action_year                     3
action_end_date               271
action_collection_year          3
action_collection              16
action_universe                 4
action_category_label           2
action_channel                  2
action_label                   14
client_is_present               2
client_is_invited               1
dtype: int64

In [19]:
merged_df['transaction_date'] = pd.to_datetime(merged_df['transaction_date'])
merged_df['action_end_date'] = pd.to_datetime(merged_df['action_end_date'])
merged_df['action_start_date'] = pd.to_datetime(merged_df['action_start_date'])

rates = []

for client_id, group in unique_transactions_df.groupby('client_id'):
    total_transactions = len(group)

    transactions_within_event_days = len(group[(group['transaction_date'] <= group['action_end_date'] + timedelta(days=7)) &
                                           (group['transaction_date'] >= group['action_start_date'] - timedelta(days=7)) &
                                           (group['client_is_invited'] == 1)])

    if total_transactions > 0:
        rate = transactions_within_event_days / total_transactions
    else:
        rate = 0

    if group['action_id'].isnull().values.any():
        rate = np.NAN

    row = {
        'client_id': client_id,
        'transactions_within_event_days': transactions_within_event_days,
        'total_transactions': total_transactions,
        'Child': 0,
        'HC': 0,
        'House': 0,
        'Jewelry': 0,
        'Men Accessory': 0,
        'Men Bags': 0,
        'Men Ready-to-Wear': 0,
        'Men Shoes': 0,
        'Men Small Leather Goods': 0,
        'Watches': 0,
        'Woman Shoes': 0,
        'Women Accessory': 0,
        'Women Bags': 0,
        'Women Ready-to-Wear': 0,
        'Women Small Leather Goods': 0,
        'rate': rate
    }

    for category, category_group in group.groupby('product_category'):
        row[category] = len(category_group)

    rates.append(row)

rates_df = pd.DataFrame(rates)
print("Stimulation rate per client_id:")
print(rates_df)


Stimulation rate per client_id:
       client_id  transactions_within_event_days  total_transactions  Child  \
0      c00029531                               0                   3      0   
1      c00055636                               0                   2      0   
2      c00068475                               0                   2      0   
3      c00089184                               1                   5      0   
4      c00091299                               0                   1      0   
...          ...                             ...                 ...    ...   
13879  c99976540                               0                   7      0   
13880  c99978675                               0                   2      0   
13881  c99989096                               0                   1      0   
13882  c99995560                               0                   1      0   
13883  c99997359                               0                   1      0   

       HC  House  J

In [20]:
rates_df[rates_df['rate'].isna()]
rates_df_original = rates_df.copy()

In [21]:
# make average rates per category as a dictionary to a dictionary
average_rates_per_category_dict = average_rates_per_category.set_index('product_category').T.to_dict('records')[0]
average_rates_per_category_dict

{'Child': 0.021945193390720624,
 'HC': 0.0,
 'House': 0.03776325344952796,
 'Jewelry': 0.07006010804230389,
 'Men Accessory': 0.06555381842659544,
 'Men Bags': 0.04696969696969697,
 'Men Ready-to-Wear': 0.04595545373274886,
 'Men Shoes': 0.07680567731588141,
 'Men Small Leather Goods': 0.03723788864786696,
 'Watches': 0.08108108108108109,
 'Woman Shoes': 0.06521509975423459,
 'Women Accessory': 0.05098850630235861,
 'Women Bags': 0.0850314809953533,
 'Women Ready-to-Wear': 0.0602817685754002,
 'Women Small Leather Goods': 0.050938664331922755}

In [22]:
missing_columns = set(average_rates_per_category_dict.keys()) - set(rates_df.columns)
print(set(rates_df.columns))
print(set(average_rates_per_category_dict.keys()))
print(missing_columns)
rates_df = rates_df.reindex(columns=rates_df.columns.tolist() + list(missing_columns))
rates_df

{'Jewelry', 'Watches', 'Woman Shoes', 'Men Small Leather Goods', 'Women Bags', 'Men Accessory', 'Men Ready-to-Wear', 'total_transactions', 'Men Bags', 'House', 'Child', 'Men Shoes', 'Women Small Leather Goods', 'Women Accessory', 'Women Ready-to-Wear', 'rate', 'transactions_within_event_days', 'HC', 'client_id'}
{'Jewelry', 'Women Accessory', 'Women Small Leather Goods', 'Watches', 'Woman Shoes', 'HC', 'Men Small Leather Goods', 'Women Bags', 'Child', 'Men Accessory', 'Men Shoes', 'Women Ready-to-Wear', 'Men Ready-to-Wear', 'Men Bags', 'House'}
set()


,client_id,transactions_within_event_days,total_transactions,Child,HC,House,Jewelry,Men Accessory,Men Bags,Men Ready-to-Wear,Men Shoes,Men Small Leather Goods,Watches,Woman Shoes,Women Accessory,Women Bags,Women Ready-to-Wear,Women Small Leather Goods,rate
0,c00029531,0,3,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.0
1,c00055636,0,2,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,NaN
2,c00068475,0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,NaN
3,c00089184,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0.2
4,c00091299,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13879,c99976540,0,7,0,0,0,0,0,0,0,0,0,0,0,6,1,0,0,0.0
13880,c99978675,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,NaN
13881,c99989096,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN
13882,c99995560,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN


In [23]:
rates_df['total_categories'] = rates_df[list(average_rates_per_category_dict.keys())].sum(axis=1)

In [24]:
rates_df

,client_id,transactions_within_event_days,total_transactions,Child,HC,House,Jewelry,Men Accessory,Men Bags,Men Ready-to-Wear,Men Shoes,Men Small Leather Goods,Watches,Woman Shoes,Women Accessory,Women Bags,Women Ready-to-Wear,Women Small Leather Goods,rate,total_categories
0,c00029531,0,3,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0.0,3
1,c00055636,0,2,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,NaN,2
2,c00068475,0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,NaN,2
3,c00089184,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0.2,5
4,c00091299,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13879,c99976540,0,7,0,0,0,0,0,0,0,0,0,0,0,6,1,0,0,0.0,7
13880,c99978675,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,NaN,2
13881,c99989096,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1
13882,c99995560,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN,1


In [25]:
for category in average_rates_per_category_dict.keys():
    rates_df[category] = rates_df[category]*average_rates_per_category_dict[category]

In [26]:
rates_df

,client_id,transactions_within_event_days,total_transactions,Child,HC,House,Jewelry,Men Accessory,Men Bags,Men Ready-to-Wear,Men Shoes,Men Small Leather Goods,Watches,Woman Shoes,Women Accessory,Women Bags,Women Ready-to-Wear,Women Small Leather Goods,rate,total_categories
0,c00029531,0,3,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.101977,0.000000,0.000000,0.050939,0.0,3
1,c00055636,0,2,0.0,0.0,0.0,0.0,0.000000,0.04697,0.000000,0.0,0.0,0.0,0.0,0.050989,0.000000,0.000000,0.000000,NaN,2
2,c00068475,0,2,0.0,0.0,0.0,0.0,0.000000,0.00000,0.091911,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,NaN,2
3,c00089184,1,5,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.301409,0.000000,0.2,5
4,c00091299,0,1,0.0,0.0,0.0,0.0,0.065554,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13879,c99976540,0,7,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.305931,0.085031,0.000000,0.000000,0.0,7
13880,c99978675,0,2,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.120564,0.000000,NaN,2
13881,c99989096,0,1,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.050989,0.000000,0.000000,0.000000,NaN,1
13882,c99995560,0,1,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.085031,0.000000,0.000000,NaN,1


In [27]:
rates_df['rate'] = rates_df[list(average_rates_per_category_dict.keys())].sum(axis=1) / rates_df['total_categories']

In [28]:
rates_df

,client_id,transactions_within_event_days,total_transactions,Child,HC,House,Jewelry,Men Accessory,Men Bags,Men Ready-to-Wear,Men Shoes,Men Small Leather Goods,Watches,Woman Shoes,Women Accessory,Women Bags,Women Ready-to-Wear,Women Small Leather Goods,rate,total_categories
0,c00029531,0,3,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.101977,0.000000,0.000000,0.050939,0.050972,3
1,c00055636,0,2,0.0,0.0,0.0,0.0,0.000000,0.04697,0.000000,0.0,0.0,0.0,0.0,0.050989,0.000000,0.000000,0.000000,0.048979,2
2,c00068475,0,2,0.0,0.0,0.0,0.0,0.000000,0.00000,0.091911,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.045955,2
3,c00089184,1,5,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.301409,0.000000,0.060282,5
4,c00091299,0,1,0.0,0.0,0.0,0.0,0.065554,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.065554,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13879,c99976540,0,7,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.305931,0.085031,0.000000,0.000000,0.055852,7
13880,c99978675,0,2,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.120564,0.000000,0.060282,2
13881,c99989096,0,1,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.050989,0.000000,0.000000,0.000000,0.050989,1
13882,c99995560,0,1,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.085031,0.000000,0.000000,0.085031,1


In [29]:
rates_df[['client_id', 'rate']].to_csv('../features/stimulation_rates.csv', index=False)